# Problem Statement

When testing complex REDCap scripts or functionalities, it is **essential** that one tests them in the Test project first before running/using them on the Target project.
Thus, there needs to be a way to make the Test project function just like the Target project.

# Solution

This notebook exports data from the Target project, then import them into the Test project. The Test project then will have the exact same records as the Target project, effectively making them the same.

### **This notebook will wipe out all existing data from the Test project before the import. The existing data will still be recoverable through a JSON file, which can later be reimported back.** 

# Instructions
Currently, the API does not allow for importing instruments. Importing instruments has to be done manually.

1. Download all instrument zips from Target project.
2. Download current Test project's instrument zips if not relevant to Target project.
3. Delete current instruments from Test project.
4. Import the Target project's instrument zips into Test project.
5. Configure repeating instruments if needed.
6. Run this notebook.



# Code 
### 1. Setup Target and Test project

In [33]:
import AMBRA_Backups
import json
import pandas as pd
from datetime import datetime

In [34]:
# TARGET PROJECT
target_project_name = 'SISTER - Image Tracking'
target_project = AMBRA_Backups.redcap_funcs.get_redcap_project(target_project_name)

# TEST PROJECT
test_project_name = 'Imaging Core - Test'
test_project = AMBRA_Backups.redcap_funcs.get_redcap_project(test_project_name)

Wipe out all existing data from `test_project`.

In [35]:
record_column = 'record_id'
all_records = test_project.export_records(format_type='df', export_blank_for_gray_form_status=True)
all_records.reset_index(inplace=True)

# Get record IDs from Test
all_records_ids = []
if record_column in all_records:
   all_records_ids = all_records['record_id'].unique()

# Before deleting records, export all current records into a json file
all_records_json = test_project.export_records(export_blank_for_gray_form_status=True)
json_obj = json.dumps(all_records_json)

with open(f'{test_project_name}_{datetime.now()}.json', 'w') as f:
   f.write(json_obj)


In [36]:
# Delete all records
if len(all_records_ids) > 0:
   test_project.delete_records(records=all_records_ids)

test_current_records = test_project.export_records(format_type='df', export_blank_for_gray_form_status=True)

if len(test_current_records) == 0:
   print(f'All records from {test_project_name} REDCap have been deleted')
else:
   raise ValueError(f'''Some records remain in {test_project_name} REDCap. current records:
         ---------
         {test_current_records}
         ''')

All records from Imaging Core - Test REDCap have been deleted


### 2. Copy records

In [37]:
'''
Get current fields from test_project to use as fields filter for
target_project's export records. 

This is needed, as when fields get deleted from a project, they 
will still show in the export_records() if there are values
in them. 
'''
# If the field is deleted, it should not show up in export_records() 
# regardless if there is a value in it
'''
Since test_project does not have data yet, it would have only the
relevant fields and not the deleted fields.
'''

test_fields_dict = test_project.export_field_names()
test_fields = set()

for field in test_fields_dict:
   test_fields.add(field['original_field_name'])

target_records = target_project.export_records(
   fields=test_fields, 
   export_blank_for_gray_form_status=True
)

# This is trying to export test fields from the target project, which does not
# have the test fields. So I get an error

target_records

RequestException: {'error': 'The following values in the parameter "fields" are not valid: \'fr509_q10q11_m2_mca\', \'fr511_q09\', \'q4089\', \'sister_image_tracking_90120_minute_solution_complete\', \'q2008\', \'fr509_q10q11_p2_pca\', \'q8095\', \'q4046\', \'hr_30_4_mrp_init\', \'fr511_q24\', \'q4012_w_lmca_lpca\', \'step_prelim_evt_complete\', \'q9106\', \'q9088\', \'q4043\', \'fr511_q04\', \'fr511_q16\', \'q8106\', \'q20cr_sig\', \'fr505_q03\', \'fr509_q0809_a3_a4_aca\', \'ms_ni_com\', \'q8087\', \'fr505_q17_m1_mca\', \'q4017_bas\', \'q8101\', \'fr509_q10q11_v4\', \'dm2mca_t\', \'q4093\', \'q2\', \'q8086\', \'q1\', \'q4008\', \'fr509_q10q11_a3_a4_aca\', \'hr_30_4_ctctactp_status\', \'pytest_text\', \'fr511_q26\', \'fr509_q0809_m2_mca\', \'q4022_int_cap\', \'a3\', \'ba\', \'ms_ni_plv\', \'q4063\', \'ellies_test_complete\', \'ndm2mca\', \'q8009\', \'q4012_raca\', \'a2_t\', \'fr505_q09\', \'fr505_add_nondom_m2_mca\', \'q4098\', \'siteid\', \'q4110\', \'fr505_add_m4_mca\', \'fr511_q17\'

Define fields that need to be taken off. 

This is necessary in cases where options were defined but got deleted from a variable. They would still retain in `export_records` if there's data in them. This is a manual process. 

Instructions:

1. Set `delete_fields = []`. Run cell.
2. If an error is thrown, copy the fields mention in the error into 
`delete_fields`.
3. Run cell again.

In [ ]:
delete_fields = [
   'sequences_acquired___ni',
   'sequences_collected___ni'
]

for record in target_records:
   for field in delete_fields:
      record.pop(field, None)

In [ ]:
# "This is necessary in cases where options were defined but got deleted from a variable"
# Do you mean to say sequences_acquired___ni was a checkbox option at one point, but was deleted later?
# Because if it were actually renamed, say "sequences_acquired___1" -> "sequences_acquired___ni",
# then the values would persist. This is because they have the same value under the option label: (1, sequences_acquired___1) == (1, sequences_acquired___ni)
# But if the new option were defined with a new option value: (1, sequences_acquired___1) == (2, sequences_acquired___ni),
# and sequences_acquired___1 were removed from the options, 
# then sequences_acquired___1's values would be deleted and not appear in export_records

# Let me know if this is what you were referring to

In [41]:
target_records = target_project.export_records(export_blank_for_gray_form_status=True)

In [42]:
delete_fields = []
for record in target_records:
   for field in delete_fields:
      record.pop(field, None)

Import data into test_project

In [43]:
test_project.import_records(target_records)

RequestException: {'error': 'The following fields were not found in the project as real data fields: notes, notes_upload, site_name, imag_comp_base, base_modalities___1, base_modalities___2, base_modalities___3, base_modalities___4, base_modalities___5, base_modalities___6, base_modalities___8, ct_date_base, ct_time_base, cta_date_base, cta_time_base, ctp_date_base, ctp_time_base, ctp_90mins, ctp_admin_window, ctp_90min_no, mri_date_base, mri_time_base, mra_date_base, mra_time_base, mrp_date_base, mrp_time_base, base_oth, follow_up_imag, follow_up_modalities___1, follow_up_modalities___2, follow_up_modalities___3, follow_up_modalities___4, follow_up_modalities___5, follow_up_modalities___6, follow_up_modalities___8, ct_date_fu, ct_time_fu, cta_date_fu, cta_time_fu, ctp_date_fu, ctp_time_fu, mri_date_fu, mri_time_fu, mra_date_fu, mra_time_fu, mrp_date_fu, mrp_time_fu, fu_other, oth_imag, oth_mod___1, oth_mod___2, oth_mod___3, oth_mod___4, oth_mod___5, oth_mod___6, oth_mod___7, ct_date_o

In [ ]:
# See if records imported
test_records_df = test_project.export_records(format_type='df', export_blank_for_gray_form_status=True)
target_records_df = target_project.export_records(format_type='df', export_blank_for_gray_form_status=True, fields=test_fields)

merged_df = pd.merge(test_records_df, target_records_df, how='outer', indicator=True)
if (len(merged_df) == len(test_records_df)) & (len(merged_df) == len(target_records_df)):
    print(f'''{test_project_name} imported data from {target_project_name} successfully''')
else:
    merged_df.to_excel(f'{test_project_name}_{target_project_name}_merged_{datetime.now()}.xlsx', header=True)
    raise ValueError(f'''Data discrepancies between {test_project_name} and {target_project_name}''')